In [4]:
import cv2
import numpy as np
import skimage
import os
from matplotlib import pyplot as plt
import pandas as pd

#霍夫变换提取直线
#https://blog.csdn.net/yukinoai/article/details/88366564
def HLines(adr):
    src = cv2.imread(adr)  #airplane l2=5417 bin=57    #forest l2=100 bin=57
    srcBlur = cv2.GaussianBlur(src, (3, 3), 0)
    gray = cv2.cvtColor(srcBlur, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    img = src.copy()
    threshold = 57
    minLineLength = 0
    maxLineGap = 0

    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold, minLineLength, maxLineGap)

    maxLength=0
    try:
        for line in lines:
            x1 = line[0][0]
            y1 = line[0][1]
            x2 = line[0][2]
            y2 = line[0][3]
            distance=(x1-x2)*(x1-x2)+(y1-y2)*(y1-y2)
            maxLength=max(distance,maxLength)
            cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    except:
        maxLength=0
    return maxLength

#主色
#https://blog.csdn.net/qq_36449923/article/details/121826842
def AVGColor(adr):
    per_image_Rmean = []
    per_image_Gmean = []
    per_image_Bmean = []
    img = cv2.imread(os.path.join(adr), 1)

    per_image_Bmean.append(np.mean(img[:,:,0]))
    per_image_Gmean.append(np.mean(img[:,:,1]))
    per_image_Rmean.append(np.mean(img[:,:,2]))

    R_mean = np.mean(per_image_Rmean)
    G_mean = np.mean(per_image_Gmean)
    B_mean = np.mean(per_image_Bmean)

    RGB=[R_mean, G_mean, B_mean]
    return RGB

#Gamma灰度
#https://blog.csdn.net/qq_42451251/article/details/107783243
def AVGGammaGray(adr):
    per_image_Rmean = []
    per_image_Gmean = []
    per_image_Bmean = []
    img = cv2.imread(os.path.join(adr), 1)

    per_image_Bmean.append(np.mean(img[:,:,0]))
    per_image_Gmean.append(np.mean(img[:,:,1]))
    per_image_Rmean.append(np.mean(img[:,:,2]))

    R_Gamma=np.power(per_image_Rmean,2.2)
    G_Gamma=np.power(per_image_Gmean,2.2)
    B_Gamma=np.power(per_image_Bmean,2.2)

    cnt=0
    Gray_Gamma=[]
    while cnt<R_Gamma.size:
        Gray_Gamma.append(pow((R_Gamma[cnt]+2.44*G_Gamma[cnt]+0.33*B_Gamma[cnt])/3.77,1.0/2.2))
        cnt+=1

    return np.mean(Gray_Gamma)

#GLCM矩阵
#https://zhuanlan.zhihu.com/p/295444415
def GLCM(adr):
    img = cv2.imread(adr, cv2.IMREAD_GRAYSCALE)
    glcm = skimage.feature.graycomatrix(img, [2, 4, 8, 16], [0, np.pi / 4, np.pi / 2, np.pi * 3 / 4], 256, symmetric = True, normed = True)
    # print(glcm.shape)

    #获取共生矩阵的统计值.
    temp = skimage.feature.greycoprops(glcm, 'contrast')
    # print(prop, temp)
    # print(prop + "_mean: ", np.mean(temp))
    # print(prop + "_std:", np.std(temp, ddof = 1))
    contrast_mean=np.mean(temp)
    contrast_std=np.std(temp, ddof = 1)

    temp = skimage.feature.greycoprops(glcm, 'dissimilarity')
    # print(prop, temp)
    # print(prop + "_mean: ", np.mean(temp))
    # print(prop + "_std:", np.std(temp, ddof = 1))


    print(temp)
    dissimilarity_mean=np.mean(temp)
    dissimilarity_std=np.std(temp, ddof = 1)

    temp = skimage.feature.greycoprops(glcm, 'ASM')
    # print(prop, temp)
    # print(prop + "_mean: ", np.mean(temp))
    # print(prop + "_std:", np.std(temp, ddof = 1))
    print(temp)
    ASM_mean=np.mean(temp)
    ASM_std=np.std(temp, ddof = 1)

    return [ASM_mean, ASM_std, contrast_mean, contrast_std, dissimilarity_mean, dissimilarity_std]




#main函数
cnt=0
features=[]
while cnt<100:
    p_lines=[]
    f_lines=[]
    adr_plane="C:/Users/14838/Desktop/MachineLearning/airplane/airplane"
    adr_forest="C:/Users/14838/Desktop/MachineLearning/forest/forest"
    if cnt<10:
        adr_plane+="0"
        adr_forest+="0"
    adr_plane+=str(cnt)
    adr_forest+=str(cnt)
    adr_plane+=".tif"
    adr_forest+=".tif"

    p_lines.append(HLines(adr_plane))
    RGB=AVGColor(adr_plane)
    for c in RGB:
        p_lines.append(c)
    p_lines.append(AVGGammaGray(adr_plane))
    GLCM_Features=GLCM(adr_plane)
    for f in GLCM_Features:
        p_lines.append(f)
    p_lines.append(1)

    f_lines.append(HLines(adr_forest))
    RGB=AVGColor(adr_forest)
    for c in RGB:
        f_lines.append(c)
    f_lines.append(AVGGammaGray(adr_forest))
    GLCM_Features=GLCM(adr_forest)
    for f in GLCM_Features:
        f_lines.append(f)
    f_lines.append(-1)

    features.append(p_lines)
    features.append(f_lines)
    cnt+=1


df = pd.DataFrame(features)
df.to_csv('features.csv')
# print(features)





C:\ProgramData\anaconda3\lib\site-packages\skimage\feature\__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
C:\ProgramData\anaconda3\lib\site-packages\skimage\feature\__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
C:\ProgramData\anaconda3\lib\site-packages\skimage\feature\__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
C:\ProgramData\anaconda3\lib\site-packages\skimage\feature\__init__.py:42: skimage_deprecation: Function ``greycoprops`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycoprops`` instead.
  removed_version='1.0')
C:\ProgramData\anaconda3\lib\site-pa